# Regression



In [22]:
from sklearn.svm import SVR
from sklearn.model_selection import LeaveOneOut, GridSearchCV
import scipy.io
import os
import numpy as np
from sklearn.metrics import mean_squared_error, make_scorer

In [2]:
path_code = os.getcwd()
bbp_dataset = scipy.io.loadmat(os.path.join(path_code, "../Data/bbp_regr.mat"))

In [19]:
ae = bbp_dataset["ae"]
conf = bbp_dataset["conf"]
wer = bbp_dataset["wer"]
y = bbp_dataset["slp"]

X = np.column_stack((ae, conf, wer))

In [17]:
loo=LeaveOneOut()

In [18]:
y_true = []
y_pred = []

In [23]:
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_neg_scorer = make_scorer(rmse_scorer, greater_is_better=False)

In [24]:
model = SVR(kernel='linear')
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5, 1]
}

In [25]:
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Modello SVR
    svr = SVR(kernel='linear')

    # Grid search con LOOCV interno per selezionare i migliori parametri
    inner_loo = LeaveOneOut()
    grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=inner_loo, scoring=rmse_neg_scorer)
    grid_search.fit(X_train, y_train)

    # Migliori parametri trovati nella grid search interna
    best_params = grid_search.best_params_
    best_svr = grid_search.best_estimator_

    # Fit del modello con i migliori parametri sul training set esterno e predizione
    best_svr.fit(X_train, y_train)
    y_pred.append(best_svr.predict(X_test)[0])
    y_true.append(y_test[0])

# Calcolare l'RMSE sui risultati finali
final_rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f'Nested LOOCV RMSE: {final_rmse:.2f}')

ValueError: Cannot perform LeaveOneOut with n_samples=1.

NameError: name 'shape' is not defined